In [91]:
import hashlib as hasher
import datetime as date

from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Signature import pkcs1_15

import random
import time


In [92]:
supply_blockchain = []
utxo_array = []
manufacturers_list = []
other_users_list = []
global_index = 0
pow_proof = int(0)

In [93]:
class Supply_Block:
    def __init__(self, index, timestamp, supply_data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.supply_data = supply_data
        self.previous_hash = previous_hash
        self.proof_of_work = int(generate_pow())
        self.hash = self.hash_block()

In [94]:
def hash_block(self):
    sha = hasher.sha256()
    sha.update((str(self.index) + 
               str(self.timestamp) + 
               str(self.supply_data) + 
               str(self.previous_hash)).encode('utf-8'))
    return sha.hexdigest()

In [95]:
def generate_pow():
    start_time = time.time()
    global pow_proof
    initial_start = pow_proof + 1
    
    while(1):
        sha = hasher.sha256()
        sha.update(str(initial_start).encode('utf-8'))
        hash_value = sha.hexdigest()
        initial_start = int(initial_start) + 1
        
        if(hash_value[0] == '0' and hash_value[1] == '0' and hash_value[2] == '0' and hash_value[-1] == '0' and hash_value[-2] == '0'):
            end_time = time.time()
            pow_proof = initial_start - 1
            print('\nThe required hash value is: ' + hash_value)
            print('The PoW number is: ' + str(pow_proof))
            print('The total time taken is: ' + str((end_time - start_time)))
            break
    return pow_proof      
            

In [96]:
class Transaction:
    def __init__(self, supplier_puk, receiver_puk, item_id, timestamp, signature):
        self.supplier_puk = supplier_puk
        self.receiver_puk = receiver_puk
        self.item_id = item_id
        self.timestamp = timestamp
        self.signature = signature

In [97]:
def create_genesis_block():
    global global_index 
    global_index = global_index + 1
    print('\n\nThe genesis block is being created.')
    return Supply_Block(0, date.datetime.now(), "GENESIS BLOCK", "0")

In [98]:
def view_blockchain():
    print('\n\nThe list of blocks are: \n')
    for block in supply_blockchain:
        print('\n------------------------------------------------------------------------------------------------------------------------')
        print(block.index)
        print(block.timestamp)
        print(block.supply_data)
        print(block.proof_of_work)
        print(block.hash)
        print(block.previous_hash)
    print('------------------------------------------------------------------------------------------------------------------------')
    print('\n\n')
    

In [99]:
def make_transaction(supplier_key, receiver_key, item_id):
    selection = input('\nSelect type of key (M/O) for supplier: ')
    if selection == 'M':
        index = int(input('There are a total of ' + str(len(manufacturers_list)) + ' users. Enter your selection: ')) - 1
        supplier_key = manufacturers_list[index]
    elif selection == 'O':
        index = int(input('There are a total of ' + str(len(other_users_list)) + ' users. Enter your selection: ')) - 1
        supplier_key = other_users_list[index]
    selection = input('\nSelect type of key (M/O) for receiver: ')  
    if selection == 'M':
        index = int(input('There are a total of ' + str(len(manufacturers_list)) + ' users. Enter your selection: ')) - 1
        receiver_key = manufacturers_list[index]
    elif selection == 'O':
        index = int(input('There are a total of ' + str(len(other_users_list)) + ' users. Enter your selection: ')) - 1
        receiver_key = other_users_list[index]
    receiver_puk = receiver_key.publickey().exportKey("PEM").decode('utf-8')
    item_id = input('Enter the ID of the tracked item: ')
    supplier_puk = supplier_key.publickey()
    timestamp = date.datetime.now()
    message = str(supplier_puk.exportKey("PEM").decode('utf-8')) + str(receiver_puk) + item_id + str(timestamp)
    hash_message = SHA256.new(message.encode('utf-8'))
    supplier_prk = RSA.import_key(supplier_key.exportKey("DER"))
    signature = pkcs1_15.new(supplier_prk).sign(hash_message)
    new_transaction = Transaction(supplier_puk, receiver_puk, item_id, timestamp, signature)
    utxo_array.append(new_transaction)

In [100]:
def mine_block():
    global global_index
    max_range = len(utxo_array)
    transaction_amount = random.randint(0, max_range)
    transaction_array = []
    print('\nThe number of selected transactions for the block is: ' + str(transaction_amount))
    if(transaction_amount):
        for index in range(0, transaction_amount):
            if(verify_transaction(utxo_array[0])):
                print('\nThe sign verification for transaction #' + str(index + 1) + ' was true!')
                if(check_item_code(utxo_array[0])):
                    print('The item code has been found. Checking the previous owner details.')
                    if(check_previous_owner(utxo_array[0])):
                        print('Verification of previous owner has been done!')
                        transaction_array.append(utxo_array[0])
                    else:
                        print('Verification of previous owner has failed!')
                else:
                    print('The item code was not found on blockchain. Checking for manufacturer credentials.')
                    if(check_manufacturer_credentials(utxo_array[0])):
                        print('The new item has been added under the manufacturer.')
                        transaction_array.append(utxo_array[0])
                    else:
                        print('The transaction key is not authorised as a manufacturer!')
            else:
                print('The sign verification for transaction #' + str(index + 1) + ' was false!')
            utxo_array.pop(0)
        if(len(transaction_array) != 0):
            new_block = Supply_Block(global_index, date.datetime.now(), transaction_array, supply_blockchain[global_index - 1].hash)
            global_index = global_index + 1
            supply_blockchain.append(new_block)
            
    else:
        print('No transactions have been selected and therefore no block has been added!')
                        

In [101]:
def verify_transaction(self):
    supplier_puk = RSA.import_key(self.supplier_puk.exportKey("DER"))
    message = str(self.supplier_puk.exportKey("PEM").decode('utf-8')) + str(self.receiver_puk) + self.item_id + str(self.timestamp)
    hash_message = SHA256.new(message.encode('utf-8'))
    try:
        pkcs1_15.new(supplier_puk).verify(hash_message, self.signature)
        return True
    except (ValueError, TypeError):
        return False
        

In [102]:
def check_item_code(self):
    found_flag = False
    temp_blockchain = supply_blockchain[::-1]
    for block in temp_blockchain[:-1:]:
        for transaction in block.supply_data:
            if(transaction.item_id == self.item_id):
                found_flag = True
    return found_flag             

In [103]:
def check_previous_owner(self):
    found_flag = False
    temp_blockchain = supply_blockchain[::-1]
    for block in temp_blockchain[:-1:]:
        for transaction in block.supply_data:
            if(transaction.item_id == self.item_id):
                if(transaction.receiver_puk == self.supplier_puk.exportKey("PEM").decode('utf-8')):
                    return True
                else:
                    return False

In [104]:
def check_manufacturer_credentials(self):
    for item in manufacturers_list:
        if str(self.supplier_puk.exportKey("PEM").decode('utf-8')) == str(item.publickey().exportKey("PEM").decode('utf-8')):
            return True
    return False    
        

In [105]:
def verify_blockchain():
    previous_block = supply_blockchain[0]
    count = 1
    for block in supply_blockchain[1:]:
        print('\nFor the block #' + str(count) + ': ')
        for transaction in block.supply_data:
            print('The item ID is ' + str(transaction.item_id) + ' and the associated timestamp is ' + str(transaction.timestamp))
        if(str(previous_block.hash) == str(block.previous_hash)):
            print('The hash values have been verified.')
        sha = hasher.sha256()
        sha.update(str(int(block.proof_of_work)).encode('utf-8'))
        hash_value = sha.hexdigest()
        print('The PoW number is ' + str(block.proof_of_work) + ' and the associated hash is ' + hash_value)
    print('------------------------------------------------------------------------------------------------------------------------') 
    print('\n\n')

In [106]:
def generate_manufacturer_keys(number):
    for item in range(0, int(number)):
        manufacturers_list.append(RSA.generate(1024, Random.new().read))
    print('\nThe manufacturer keys have been generated.')
    

In [107]:
def generate_other_keys(number):
    for item in range(0, int(number)):
        other_users_list.append(RSA.generate(1024, Random.new().read))
    print('\nThe stakeholder keys have been generated.')
        

In [108]:
def track_item(item_code):
    not_found_flag = True
    for block in supply_blockchain[1:]:
        for transaction in block.supply_data:
            if(item_code == transaction.item_id):
                if(not_found_flag):
                    print('\nThe item (' + item_code + ') has been found and the tracking details are: ')
                    not_found_flag = False
                manufacturer_suppplier = False 
                manufacturer_receiver = False
                supplier_count = 0
                supplier_not_found_flag = True
                for item in manufacturers_list:
                    supplier_count = supplier_count + 1
                    if str(transaction.supplier_puk.exportKey("PEM").decode('utf-8')) == str(item.publickey().exportKey("PEM").decode('utf-8')):
                        supplier_not_found_flag = False
                        manufacturer_suppplier = True
                        break
                if(supplier_not_found_flag):
                    supplier_count = 0
                    for item in other_users_list:
                        supplier_count = supplier_count + 1
                        if str(transaction.supplier_puk.exportKey("PEM").decode('utf-8')) == str(item.publickey().exportKey("PEM").decode('utf-8')):
                            supplier_not_found_flag = False
                            break
                receiver_count = 0 
                receiver_not_found_flag = True
                for item in manufacturers_list:
                    receiver_count = receiver_count + 1
                    if str(transaction.receiver_puk) == str(item.publickey().exportKey("PEM").decode('utf-8')):
                        receiver_not_found_flag = False
                        manufacturer_receiver = True
                        break
                if(receiver_not_found_flag):
                    receiver_count = 0
                    for item in other_users_list:
                        receiver_count = receiver_count + 1
                        if str(transaction.receiver_puk) == str(item.publickey().exportKey("PEM").decode('utf-8')):
                            receiver_not_found_flag = False
                            break
                final_result = ""
                if(manufacturer_suppplier):
                    final_result = final_result + "Manufacturer #" + str(supplier_count) + " transferred the asset to "
                else:
                    final_result = final_result + "Stakeholder #" + str(supplier_count) + " transferred the asset to "
                    
                if(manufacturer_receiver):
                    final_result = final_result + "Manufacturer #" + str(receiver_count) + " at " + str(transaction.timestamp)
                else:
                    final_result = final_result + "Stakeholder #" + str(receiver_count) + " at " + str(transaction.timestamp)
                print(final_result)
       
    if(not_found_flag):
        print('\nThe item code was not found in the blockchain.')
        


In [ ]:
number_manufacturers = int(input('\nEnter the number of manufacturers: '))
generate_manufacturer_keys(number_manufacturers)

number_other_users = int(input('\nEnter the number of stakeholders: '))
generate_other_keys(number_other_users)

supply_blockchain.append(create_genesis_block())
print('\n\nWelcome to the supply blockchain.')

while(1):
    print('\nThe following options are available to the user: ')
    print('1. View the blockchain. ')
    print('2. Enter a transaction. ')
    print('3. View the UTXO array. ')
    print('4. Mine a block. ')
    print('5. Verify the blockchain. ')
    print('6. Generate RSA keys. ')
    print('7. Track an item.')
    print('8. Exit.')
    
    choice = int(input('Enter your choice: '))
    if(choice == 1):
        view_blockchain()
    elif(choice == 2):
        make_transaction('','','')
    elif(choice == 3):
        view_UTXO()
    elif(choice == 4):
        mine_block()
    elif(choice == 5):
        verify_blockchain()
    elif(choice == 6):
        number_manufacturers = int(input('\nEnter the number of manufacturers: '))
        generate_manufacturer_keys(number_manufacturers)
        number_other_users = int(input('Enter the number of stakeholders: '))
        generate_other_keys(number_other_users)
    elif(choice == 7):
        item_code = input('Enter the item code: ')
        track_item(item_code)
    elif(choice == 8):
        break
    else:
        print('This is an invalid option.')
            
        
        
        
            
                    
                        